In [ ]:
%load_ext autoreload
%autoreload 2

import os,sys
import mne
import utils  #my code
import json
import matplotlib.pyplot as plt
import numpy as np
import utils_preproc as upre




#rawname_ = 'S01_off_hold'
rawname_ = 'S01_off_move'
#rawname_ = 'S01_on_hold'
#rawname_ = 'S01_on_move'

#rawname_ = 'S02_off_hold'
#rawname_ = 'S02_on_hold' 
#rawname_ = 'S02_off_move'  
#rawname_ = 'S02_on_move'

#rawname_ = 'S03_off_move'  
#rawname_ = 'S03_off_hold'  

#rawname_ = 'S04_off_hold'
#rawname_ = 'S04_off_move'
#rawname_ = 'S04_on_hold'
#rawname_ = 'S04_on_move'

#rawname_ = 'S05_off_hold'
#rawname_ = 'S05_off_move'
#rawname_ = 'S05_on_hold'
#rawname_ = 'S05_on_move'

#rawname_ = 'S07_off_hold'
#rawname_ = 'S07_off_move'
#rawname_ = 'S07_on_hold'
#rawname_ = 'S07_on_move'

import globvars as gv


rawname = rawname_ + '_resample_raw.fif'
#rawname = rawname_ + '_notch_SSS_raw.fif'
#rawname = rawname_ + '_resample_afterICA_raw.fif'
fname_full = os.path.join(gv.data_dir,rawname)

raw = mne.io.read_raw_fif(fname_full, None)

In [ ]:
sind_str,mod,task = utils.getParamsFromRawname(rawname_)

sources_type='parcel_aal'
src_file_grouping_ind = 10
src_rec_info_fn = '{}_{}_grp{}_src_rec_info'.format(rawname_,
                                                    sources_type,src_file_grouping_ind)
src_rec_info_fn_full = os.path.join(gv.data_dir, src_rec_info_fn + '.npz')
rec_info = np.load(src_rec_info_fn_full, allow_pickle=True)


print( list(rec_info.keys()) )

labels_dict = rec_info['label_groups_dict'][()]
srcgroups_dict = rec_info['srcgroups_dict'][()]
coords = rec_info['coords_Jan_actual'][()]

print( labels_dict.keys() )

sgdn = 'all_raw'

In [ ]:
display(chan_info)


In [ ]:
#raw.pick_channels(raw.ch_names)

In [ ]:
meg_chis = mne.pick_channels_regexp(raw.ch_names,'MEG.*')

chlocs = []
for si in range(len(meg_chis)):
    chi = meg_chis[si]
    chan_info = raw.info['chs'][chi]
    chan_loc = chan_info['loc'][:3]
    chlocs += [chan_loc]
chlocs = np.vstack(chlocs)

#x,y,z = chlocs.T*100
# another run -- check which distances are small then the average one

In [ ]:
#x*100

In [ ]:
x,y,z = chlocs.T*100

In [ ]:
utils.vizGroup(sind_str,coords,labels_dict[sgdn],srcgroups_dict[sgdn], show=False)

import mayavi.mlab as mam
mam.points3d(x,y,z, scale_factor=0.5)#, color = tuple(clrs[grpi]) )

mam.show()

In [ ]:
lbls = ['Supp_Motor_Area_L', 'Cerebellum_R']
#lbls = [ 'Cerebellum_R']


import utils_test

roi_sensors = utils_test.getSensorsFromRoiLabels(lbls, coords, chlocs*100, 
                                                 labels_dict[sgdn], 
                                                 srcgroups_dict[sgdn] )

In [ ]:
roi_sensors

In [ ]:
# TODO: for each key select corresponding sources and found sensors and plot both
sensors_inds = []
src_inds = []
for roi_label in roi_sensors:
    sensors_inds +=  list(roi_sensors[roi_label] ) 
    
    src_inds += list(np.where(srcgroups_dict[sgdn] == labels_dict[sgdn].index(roi_label))[0])


x,y,z = chlocs.T * 100
mam.points3d(x,y,z, scale_factor=0.3, color=(0.1,0.1,0.1))#, color = tuple(clrs[grpi]) )
    
x,y,z = chlocs[sensors_inds].T * 100
mam.points3d(x,y,z, scale_factor=0.5)#, color = tuple(clrs[grpi]) )

x,y,z = coords[src_inds].T 
mam.points3d(x,y,z, scale_factor=0.5, color=(1,1,0))#, color = tuple(clrs[grpi]) )


mam.show()

In [ ]:
best_dists

In [ ]:
getSe

In [ ]:
#best_inds

In [ ]:
best_dist_mean = np.mean(best_dists)
print(best_dist_mean)

In [ ]:
print(dist_mean)

In [ ]:
np.min(dists),np.max((dists))

In [ ]:
# (o,oo) = np.unravel_index(8199, dists_loc.shape)

# print(source_inds[o],oo)

# dists_loc[o,oo]


In [ ]:
roi_sensors

In [ ]:
assert len(corresp_sensors) < len(meg_chis), '{} -- Too many :('.\
    format(len(corresp_sensors))

In [ ]:
#corresp_sensors

In [ ]:
import mayavi.mlab as mam

x,y,z = chlocs.T*90
mam.points3d(x,y,z, scale_factor=0.5)#, color = tuple(clrs[grpi]) )

mam.show()

In [ ]:
# TODO: update due to update of MNE
mod_info, infos = upre.readInfo(rawname_, raw)  
radius, origin, _ = mne.bem.fit_sphere_to_headshape(mod_info, dig_kinds=('cardinal','hpi'))

sphere = mne.make_sphere_model(info=mod_info, r0=origin, head_radius=radius)
#sphere = mne.make_sphere_model(info=mod_info, r0='auto', head_radius='auto')
#sphere = mne.make_sphere_model(info=raw_tmp.info, r0='auto', head_radius='auto')
src = mne.setup_volume_source_space(sphere=sphere, pos=10.)

plot_sensor_helmet = False
if plot_sensor_helmet:
    mne.viz.plot_alignment(
        mod_info, eeg='projected', bem=sphere, src=src, dig=True,
        surfaces=['brain', 'outer_skin'], coord_frame='meg', show_axes=True)

print('Origin is ', origin)